## 1. Load libraries

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
import warnings
warnings.simplefilter('ignore')
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.graph_objs as go
import plotly 
from plotly import tools
from plotly.graph_objs import *
import collections
import calendar
%matplotlib inline 
import datetime
import colorlover as cl
import joypy
from matplotlib import cm
import plotnine as pln
from matplotlib.colors import ListedColormap
import matplotlib


plt.style.use('ggplot')
plt.figure(figsize=(18,10))

import seaborn as sns
import os
import folium
from folium.plugins import MarkerCluster
print(folium.__version__)
cmap = ListedColormap(sns.color_palette())

import colorlover as cl
from IPython.display import HTML

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go

init_notebook_mode(connected=True)

from sklearn.preprocessing import LabelEncoder
import ggplot
from ggplot import *
matplotlib.rcParams['figure.figsize'] = (20.0, 10.0)
# plotly.tools.set_credentials_file(username='adslwang4601', api_key='RNtOcvlYrBgpU80Tiiuf')


0.5.0


## 2. Load data

In [3]:
path = '../input/'
# 训练集
train = pd.read_table(path+'round1_ijcai_18_train_20180301.txt',encoding='utf8',delim_whitespace=True)
train['isTrain'] = 1
    
# 测试集
test = pd.read_table(path+'round1_ijcai_18_test_a_20180301.txt',encoding='utf8',delim_whitespace=True)
test['isTrain'] = 0
        
# 连接
df = pd.concat([train, test])
print("========> Load Data Success!") 

========> Load Data Success!


## 3. Glimpse at the dataset

### 3.1 Train

In [4]:
train.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,is_trade,isTrain
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,1
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,1
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,1
3,937088850059189027,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,509660095530134768:-1;5799347067982556520:-1;7...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,1
4,7975697065017708072,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0,0,1


### 3.2 Test

In [5]:
test.head()

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,isTrain
0,2475218615076601065,2275895163219263378,7908382889764677758;8277336076276184272,2636395404473730413;7515802706813700848;302719...,7838285046767229711,7534238860363577544,7,6,8,14,...,4005,8277336076276184272:-1;5799347067982556520:-1;...,5275548996133547595,13,0.985244,5011,0.964023,0.966550,0.955378,0
1,398316874173557226,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,4001,"5755694407684602296:216555019471416386,2636395...",329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,0
2,6586402638209028583,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,4011,5755694407684602296:2636395404473730413;871073...,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,0
3,1040996105851528465,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,4001,8710739180200009128:-1;5755694407684602296:263...,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,0
4,6316278569655873454,7096238490711246967,7908382889764677758;5755694407684602296,5131280576272319091;2636395404473730413;734498...,8126195666233054089,3948283326616421003,6,9,8,16,...,4001,5755694407684602296:5131280576272319091;871073...,329470548383752413,14,0.989068,5012,0.963794,0.964225,0.959134,0


### 3.3 Missing values

In [6]:
train.isnull().sum()

instance_id                  0
item_id                      0
item_category_list           0
item_property_list           0
item_brand_id                0
item_city_id                 0
item_price_level             0
item_sales_level             0
item_collected_level         0
item_pv_level                0
user_id                      0
user_gender_id               0
user_age_level               0
user_occupation_id           0
user_star_level              0
context_id                   0
context_timestamp            0
context_page_id              0
predict_category_property    0
shop_id                      0
shop_review_num_level        0
shop_review_positive_rate    0
shop_star_level              0
shop_score_service           0
shop_score_delivery          0
shop_score_description       0
is_trade                     0
isTrain                      0
dtype: int64

### 3.4 File info

In [7]:
rows_train = train.shape[0]
print('Number of rows in the train set: %d' % rows_train)
size_train = os.path.getsize(path+'round1_ijcai_18_train_20180301.txt')/(1024*1024)
print('Train set file size: %d MB' % size_train)
rows_test= test.shape[0]
print('Number of rows in the test set: %d' % rows_test)
size_test = os.path.getsize(path+'round1_ijcai_18_test_a_20180301.txt')/(1024*1024)
print('Test set file size: %d MB' % size_test)

Number of rows in the train set: 478138
Train set file size: 529 MB
Number of rows in the test set: 18371
Test set file size: 20 MB


### 3.5 Unique counts

In [ ]:
train.nunique()

instance_id                  478087
item_id                       10075
item_category_list               14
item_property_list            10908
item_brand_id                  2055
item_city_id                    128
item_price_level                 14
item_sales_level                 18
item_collected_level             18
item_pv_level                    22
user_id                      197694
user_gender_id                    4
user_age_level                    9
user_occupation_id                5
user_star_level                  12
context_id                   478111
context_timestamp            281824
context_page_id                  20
predict_category_property     78796
shop_id                        3959
shop_review_num_level            25
shop_review_positive_rate     11825
shop_star_level                  22
shop_score_service            16361
shop_score_delivery           16423
shop_score_description        16463
is_trade                          2
isTrain                     

## 4. Features visualization
Here we have a first look at the distributions of the features in our individual data files before combining them for a more detailed analysis. This visualization will be the foundation on which we build our analysis.

### 4.1 The information of advertising  products

### 4.1.1 item_id
It seems that view count has no correlation with trade_rate. As shown in Figure2, numbers of most items are less 500. 

In [ ]:
item_data = train[['item_id', 'item_category_list', 'item_property_list', 'item_brand_id', 'item_city_id', \
                  'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 'is_trade']]
item_count = item_data['item_id'].value_counts()[:50].reset_index()
item_count.columns = ['item_id', 'count']

top_50_data = item_data.merge(item_count, on=['item_id'])
popular = top_50_data.groupby(by=['item_id']).is_trade.agg(['mean']).reset_index()
popular.columns = ['item_id', 'trade_rate']
merge_count = item_count.merge(popular, on=['item_id'])

fig, axes = plt.subplots(2, 1, figsize=(12,8))
ax_y = axes[0].twinx()
sns.barplot(x = list(range(item_count.shape[0])), y=merge_count['count'], ax=axes[0])
sns.pointplot(x = list(range(item_count.shape[0])), y=merge_count['trade_rate'], ax=ax_y)
axes[0].set_title('Trade ratess over counts of 50 most popular items')

axes[1].scatter(x=list(range(item_data['item_id'].value_counts().shape[0])), \
                y=item_data['item_id'].value_counts().values, s=2)
axes[1].set_title('The distribution of item count')
axes[1].set_ylabel('count')



Text(0,0.5,'count')

## 4.1.2 item_category_list

In [ ]:
def stack_bar(column, num_categories):
    le = LabelEncoder()
    item_data[column] = le.fit_transform(item_data[column])
    item_data[column] = item_data[column].apply(lambda x: '%s_' %column +str(x))
    item_data_subset = item_data[column].value_counts()[:num_categories].reset_index()
    item_data_subset.columns = ['%s' % column, '%s_count' % column]

    item_data_subset = item_data_subset.merge(item_data, how='inner', on=column)
    count_cate1 = item_data_subset[item_data_subset['is_trade']==1][column].value_counts().sort_index()
    count_cate0 = item_data_subset[item_data_subset['is_trade']==0][column].value_counts().sort_index()
    
    
    trace1 = go.Bar(
        x=count_cate0.index,
        y=count_cate0.values,
        name='is_trade==0'
    )
    trace2 = go.Bar(
        x=count_cate1.index,
        y=count_cate1.values,
        name='is_trade==1'
    )

    data = [trace1, trace2]
    layout = go.Layout(
        barmode='stack'
    )

    fig = go.Figure(data=data, layout=layout)
    return fig

In [ ]:
item_data['item_category_list']
for i in range(3):
        item_data['category_%d' % (i)] = item_data['item_category_list'].apply(
            lambda x: x.split(";")[i] if len(x.split(";")) > i else " "
        )

In [ ]:
fig = plt.figure(figsize=(15, 5))
ax1 = fig.add_subplot(1,3,1)
item_data['category_0'].value_counts().plot.bar(ax=ax1)
ax2 = fig.add_subplot(1,3,2)
item_data['category_1'].value_counts().plot.bar(ax=ax2)
ax3 = fig.add_subplot(1,3,3)
item_data['category_2'].value_counts().plot.bar(ax=ax3)

In [ ]:
le = LabelEncoder()
item_data['category_1'] = le.fit_transform(item_data['category_1'])
count_cate1 = item_data[item_data['is_trade']==1]['category_1'].value_counts().sort_index()
count_cate0 = item_data[item_data['is_trade']==0]['category_1'].value_counts().sort_index()
trace1 = go.Bar(
    x=count_cate0.index,
    y=count_cate0.values,
    name='is_trade==0'
)
trace2 = go.Bar(
    x=count_cate1.index,
    y=count_cate1.values,
    name='is_trade==1'
)

data = [trace1, trace2]
layout = go.Layout(
    barmode='stack'
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='stacked-bar')

### 4.1.3 item_brand_id

In [ ]:
column = 'item_brand_id'
fig = stack_bar(column, 15)
py.iplot(fig, filename='stacked-%s_bar' % column)

In [ ]:
item_data['item_brand_id'].value_counts()[:50].plot.bar()

### 4.1.4 item_city_id

In [ ]:
column = 'item_city_id'
fig = stack_bar(column, 50)
py.iplot(fig, filename='stacked-%s_bar' % column)

In [ ]:
item_data['item_brand_id'].value_counts()[:50].plot.bar()

### 4.1.5 item_price_level

In [ ]:
item_data['item_price_level'].value_counts().plot.bar()

In [ ]:
item_data['item_price_level'].value_counts()